In [1]:
import json
import requests
import os
import time

In [2]:
import psycopg2
import os
import json

database_dict = {
        "database": os.environ.get("POSTGRES_DB"),
        "user": os.environ.get("POSTGRES_USERNAME"),
        "password": os.environ.get("POSTGRES_PASSWORD"),
        "host": os.environ.get("POSTGRES_WRITER"),
        "port": os.environ.get("POSTGRES_PORT"),
    }
engine = psycopg2.connect(**database_dict)
cur = engine.cursor()

def get_all_members(cur, engine):
    execute_string = f"SELECT * FROM daohaus.member"
    cur.execute(execute_string)
    records = cur.fetchall()
    js = [];
    for x in records:
        curl = []
        curl.append(x[1])
        curl.append(x[6])
        js.append(curl)
    return js

def get_all_daos_chain(cur, engine):
    execute_string = f"SELECT * FROM daohaus.dao"
    cur.execute(execute_string)
    records = cur.fetchall()
    js = {};
    for x in records:
        js[x[0]] = x[5]
    return js

all_members = get_all_members(cur, engine)
all_daos_chains = get_all_daos_chain(cur, engine)

In [3]:
xDai_members = []
for member in all_members:
    if(all_daos_chains[member[0]] == 392):
        xDai_members.append(member[1])

In [4]:
md_of_all_members_tokens_and_balances = {}

In [5]:
all_tokens = set()

In [6]:
for member in xDai_members:
    response = requests.get(f"https://blockscout.com/xdai/mainnet/api?module=account&action=tokenlist&address={member}")
    test_data = response.json()
    if(test_data["message"] == "OK"):
        list_tokens_and_data = []
        for token in test_data["result"]:
            cd = {}
            cd['contractAddress'] = token['contractAddress']
            cd['balance'] = token['balance']
            cd['divisor'] = token['decimals']
            cd['name'] = token['name']
            list_tokens_and_data.append(cd)
            all_tokens.add(token['contractAddress'])
        
        md_of_all_members_tokens_and_balances[str(member)] = list_tokens_and_data
    elif (test_data["message"] == "No tokens found"):
        md_of_all_members_tokens_and_balances[str(member)] = "NO TOKENS"
    time.sleep(1)
    print(member, test_data["message"])

0xced608aa29bb92185d9b6340adcbfa263dae075b OK
0xd1629474d25a63b1018fcc965e1d218a00f6cbd3 OK
0xd3a4f5d6b87a3254fe28965c716e90f5e5362291 OK
0x29ae8ab8afd0d3ca1a58934398bed2f9347a8e95 OK
0x3dbb1c5cbfbc0235d04b96603724efbf49db5f04 OK
0x7496aacfb61a57e3621c14eaa025158e66ce46b5 No tokens found
0xeb94bd011f6b794b6c846457b85efa895d2dd744 OK
0x16765c8fe6eb838cb8f64e425b6dccab38d4f102 OK
0x0b1793f79dd7402c6695b567e9879179857b577e No tokens found
0x1b17e906edc0af2f658419bbea4a39a10506886e OK
0x2d2068efd6eda370240378dd7ae021ece060d27d No tokens found
0x377a35af928301f90ab76e4a8384e6fb51da06db No tokens found
0x39736a864b347a7e7493324dad7583e469e29bf3 No tokens found
0x4b3a1e477159b528df50ed032744bff7cbdb488d No tokens found
0x51fd04e8c5354dd28dcb0f9499dd8e612200480a No tokens found
0xc12e5182d3aaec84cc6bf00863e2aeaa5b4f326f No tokens found
0xd61abd85571f02bddc8251c3dc3f31f0657cc1bc OK
0xe832259427f518a8a86b98d4963fff7386bdf270 OK
0xf048b6ca5cdcc36fb80136bbfdae9f07058d46cd No tokens found
0xf39bd34

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
js_str = json.dumps(md_of_all_members_tokens_and_balances)
js_fin = json.loads(js_str)
with open('./all_xDai_member_with_token_balances.json', 'w') as f:
        json.dump(js_fin, f)